<b> Source COVID World updated everyday at 2:00 AM : </b> <br> 
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series <br>

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [5]:
#--- STEP1 ---: Get data source & cleaning
start_time = time.time()

#-- GITHUB : TIMESERIES (CSSEGISandData/COVID-19/csse_covid_19_time_series)
#-> WORLD_CONFIRMED
urlConfirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
cConfirmed=check_url(urlConfirmed)
if use_mongo:
    bob.mongo.save_df(cConfirmed,'WORLD_CONFIRMED',db_src,True)

#-> WORLD_DEATHS
urlDeath = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
cDeath=check_url(urlDeath)
if use_mongo:
    bob.mongo.save_df(cDeath,'WORLD_DEATHS',db_src,True)

#-> WORLD_RECOVERED
urlRecovery = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
cRecovery=check_url(urlRecovery)
if use_mongo:
    bob.mongo.save_df(cRecovery,'WORLD_RECOVERED',db_src,True)

#-> US_CONFIRMED
urlConfirmedUS = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
cConfirmedUS=check_url(urlConfirmedUS)
if use_mongo:
    bob.mongo.save_df(cConfirmed,'US_CONFIRMED',db_src,True)
cConfirmedUS.to_csv(output_folder + 'US_CONFIRMED.csv',sep=";") 

#-> US_DEATHS
urlDeathUS = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
cDeathUS=check_url(urlDeathUS)
if use_mongo:
    bob.mongo.save_df(cDeath,'US_DEATHS',db_src,True)
cDeathUS.to_csv(output_folder + 'US_DEATHS.csv',sep=";") 

#-- GITHUB : DAILY REPORT (CSSEGISandData/COVID-19/csse_covid_19_time_series)
#-> DAILY
date_init = yesterday.strftime('%m-%d-%Y')
date_end = yesterday.strftime('%m-%d-%Y')
dates = pd.date_range(start=date_init,end=date_end, freq='D').strftime('%m-%d-%Y').values.tolist()
filter_dates = pd.date_range(start=date_init,end=date_end, freq='D').strftime('%m/%d/%y').values.tolist()

cDaily = pd.DataFrame()
for d in dates:
    urlDaily = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + d + '.csv'
    cols_to_rename = {"Country/Region": "Country_Region","Province/State": "Province_State", "Long_":"Long"}
    tmp_df = check_url(urlDaily).rename(index=str, columns=cols_to_rename)
    if len(tmp_df) != 0:
        tmp_df['Confirmed'] = tmp_df['Confirmed'].fillna(0).astype(int)
        tmp_df['Deaths'] = tmp_df['Deaths'].fillna(0).astype(int)
        tmp_df['Recovered'] = tmp_df['Recovered'].fillna(0).astype(int)
        tmp_df['Date'] = pd.to_datetime(d, format='%m-%d-%Y').strftime('%m/%d/%y')
        cDaily = cDaily.append(tmp_df)
        
with pd.ExcelWriter(output_folder + 'WSR_datasource_' + yesterday.strftime('%Y%m%d') + '.xlsx') as writer:  
    cConfirmed.to_excel(writer,sheet_name ="WORLD_CONFIRMED")
    cDeath.to_excel(writer,sheet_name ="WORLD_DEATHS")
    cRecovery.to_excel(writer,sheet_name ="WORLD_RECOVERED")
    cDaily.to_excel(writer,sheet_name ="WORLD_DAILY")
    cConfirmedUS.to_excel(writer,sheet_name ="US_CONFIRMED")
    cDeathUS.to_excel(writer,sheet_name ="US_DEATHS")
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Dataframe WORLD_CONFIRMED successfully save in database covid-19-dev in MongoDB. Time: --- 0.0682826042175293 secnds ---
Dataframe WORLD_DEATHS successfully save in database covid-19-dev in MongoDB. Time: --- 0.06247282028198242 secnds ---
Dataframe WORLD_RECOVERED successfully save in database covid-19-dev in MongoDB. Time: --- 0.07368993759155273 secnds ---
Dataframe US_CONFIRMED successfully save in database covid-19-dev in MongoDB. Time: --- 0.062105417251586914 secnds ---
Dataframe US_DEATHS successfully save in database covid-19-dev in MongoDB. Time: --- 0.06197047233581543 secnds ---
Script execution completed at 13/04/2020 22:42:15. Time: --- 27.603705406188965 secnds ---


In [11]:
#-- Melt data source from timeseries to have date in rows
def melt_cleaning(df,variable):
    try:
        if 'Province/State' and 'Country/Region' in df.columns:
            #Add Country/Region to Province/State if empty
            df = df.fillna(0)
            df.loc[df['Province/State'] == 0,'Province/State'] = df['Country/Region']
            #Melt and group data
            cols_to_keep = ['Country/Region','Province/State']
            cols_to_group = ['Country/Region','Province/State','Date']
            df= df.melt(id_vars=cols_to_keep,value_name=variable,var_name='Date').groupby(cols_to_group, as_index=False).agg({variable:'sum'})
            #Data format
            df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')

            if variable == 'Confirmed':
                df['Deaths'] = 0
                df['Recovered'] = 0
            elif variable == 'Deaths':
                df['Confirmed'] = 0
                df['Recovered'] = 0
            else:
                df['Confirmed'] = 0
                df['Deaths'] = 0
        else:
            df=pd.DataFrame()
    except Exception as e:
        df=pd.DataFrame()
        print(e.__doc__)
        print(str(e))  
    return df

def cols_cleaning(df,cols_to_drop):
    for c in cols_to_drop:
        if c in df.columns:
            df = df.drop(c,axis=1)
            
    cols_to_rename = {"Province_State": "Province/State", "Country_Region": "Country/Region"}
    df = df.rename(index=str, columns=cols_to_rename)
    return df       

# Timeseries : World Confirmed
cols_to_drop = ['Lat','Long']
cConfirmed=cConfirmed[cConfirmed['Country/Region'] != 'US']
df_cW = melt_cleaning(cols_cleaning(cConfirmed, cols_to_drop),'Confirmed')
    
# Timeseries : World Deaths
cols_to_drop = ['Lat','Long']
cDeath=cDeath[cDeath['Country/Region'] != 'US']
df_dW = melt_cleaning(cols_cleaning(cDeath, cols_to_drop),'Deaths')
    
# Timeseries : World Recovered
cols_to_drop = ['Lat','Long']
df_rW = melt_cleaning(cols_cleaning(cRecovery, cols_to_drop),'Recovered')

# Timeseries : US Confirmed
cols_to_drop = ['UID','iso2', 'iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key']
df_cUS = melt_cleaning(cols_cleaning(cConfirmedUS, cols_to_drop),'Confirmed')

# Timeseries : US Deaths
cols_to_drop = ['UID','iso2', 'iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key','Population']
df_dUS = melt_cleaning(cols_cleaning(cDeathUS, cols_to_drop),'Deaths')

# Manual input
df_manual = pd.read_excel(input_folder + 'REF_WSR.xlsx', sheet_name = 'DATA_INPUT').fillna(0)
df_manual['Date'] = pd.to_datetime(df_manual['Date'], format='%Y%m%d').dt.strftime('%m/%d/%y')
# df_manual

In [12]:
#--- STEP2 ---: Create DB with data source (output = DB_CONCAT)
start_time = time.time()
def step2(df1,df2,df3,df4,df5,df6):
    #-- Merge timeseries
    cols_to_group = ['Country/Region','Province/State','Date']
    df = pd.concat([df1,df2,df3,df4,df5,df6], axis=0).groupby(cols_to_group, as_index=False).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'})
    return df.reset_index(drop=True)

db_concat = step2(df_cW,df_dW,df_rW,df_cUS,df_dUS,df_manual)
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
db_concat.to_csv(output_folder + 'WORLD_DB_CONCAT.csv',sep=";")
# bob.mongo.save_df(db_concat,'WORLD_DB_CONCAT',db_src,True)
# db_concat

Script execution completed at 13/04/2020 12:25:31. Time: --- 0.054505348205566406 secnds ---
Dataframe WORLD_DB_CONCAT successfully save in database covid-19-dev in MongoDB. Time: --- 0.7700128555297852 secnds ---


,Country/Region,Province/State,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,01/22/20,0,0,0.0
1,Afghanistan,Afghanistan,01/23/20,0,0,0.0
2,Afghanistan,Afghanistan,01/24/20,0,0,0.0
3,Afghanistan,Afghanistan,01/25/20,0,0,0.0
4,Afghanistan,Afghanistan,01/26/20,0,0,0.0
...,...,...,...,...,...,...
26481,Zimbabwe,Zimbabwe,04/08/20,11,3,0.0
26482,Zimbabwe,Zimbabwe,04/09/20,11,3,0.0
26483,Zimbabwe,Zimbabwe,04/10/20,13,3,0.0
26484,Zimbabwe,Zimbabwe,04/11/20,14,3,0.0


In [13]:
#--- STEP3 ---: Consolidate and enrich data (output = DB_CONSO)
start_time = time.time()
#-> Referentials
ref_continent = pd.read_excel(input_folder + 'REF_WSR.xlsx', sheet_name = 'REF_CONTINENT')
def step3(df,ref):
    df.columns = df.columns.str.upper()
    cols_to_rename = {"COUNTRY/REGION": "ENTITY_GROUPS", "PROVINCE/STATE": "ENTITY"}
    df = df.rename(index=str, columns=cols_to_rename)
    
    cols_to_rename = {"ENTITY_GROUPS": "ENTITY", "CONTINENT": "ENTITY_GROUPS"}
    df_con = pd.merge(df.drop(['ENTITY'],axis=1),ref.drop(['WORLDMAP'],axis=1), left_on=['ENTITY_GROUPS'], right_on=['COUNTRY_REGION'], how='left').rename(index=str, columns=cols_to_rename).drop(['COUNTRY_REGION'],axis=1).fillna(0)
    df_con.loc[df_con['ENTITY_GROUPS'] == 0, 'ENTITY_GROUPS'] = 'To be affected'
    
    cols_to_rename = {"ENTITY_GROUPS": "ENTITY"}
    df_ww = df_con.copy().drop(['ENTITY'],axis=1).rename(index=str, columns=cols_to_rename)
    df_ww['ENTITY_GROUPS'] = 'WORLDWIDE'
    
    cols_to_rename = {"ENTITY_GROUPS": "ENTITY"}
    df_tt = df_ww.copy().drop(['ENTITY'],axis=1).rename(index=str, columns=cols_to_rename)
    df_tt['ENTITY_GROUPS'] = 'WORLDWIDE'

    df = pd.concat([df,df_con,df_ww,df_tt], axis=0)
    df = df.groupby(['ENTITY_GROUPS','ENTITY','DATE'], as_index=False).agg({'CONFIRMED': 'sum','DEATHS': 'sum', 'RECOVERED': 'sum'})
    return df.reset_index(drop=True)

db_conso = step3(db_concat,ref_continent)
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
db_conso.to_csv(output_folder + 'WORLD_DB_CONSO.csv',sep=";")
# if use_mongo:
#     bob.mongo.save_df(db_conso,'WORLD_DB_CONSO',db_src,True)
# db_conso

Script execution completed at 13/04/2020 12:25:32. Time: --- 0.2988009452819824 secnds ---
Dataframe WORLD_DB_CONSO successfully save in database covid-19-dev in MongoDB. Time: --- 30.186661958694458 secnds ---


In [14]:
#--- STEP4 ---: Calculate KPIs (output = DB_ALL)
start_time = time.time()
#-> Referentials
ref_tech = pd.read_excel(input_folder + 'REF_WSR.xlsx', sheet_name = 'PARAM_ALL')

def step4(df,ref):
    #-- Calc Active + Ratio
    df['ACTIVE_CASES'] = df['CONFIRMED'] - df['DEATHS'] - df['RECOVERED']
    df.loc[:,'DEATHS_RATIO'] = 0
    df.loc[df['CONFIRMED'] != 0, 'DEATHS_RATIO'] = df['DEATHS'] / df['CONFIRMED'] * 100
    df.loc[:,'RECOVERED_RATIO'] = 0
    df.loc[df['CONFIRMED'] != 0, 'RECOVERED_RATIO'] = df['RECOVERED'] / df['CONFIRMED'] * 100
    
    #-- Melt KPI in rows
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','DATE']
    df= df.melt(id_vars=cols_to_keep,value_name='VALUE',var_name='KPI')
    indexes = df.loc[df['DATE'] == 0].index
    df = df.drop(indexes, axis=0)
    
    #-- Variation vs last day
    #Add fields
    df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%y')
    df['LAST_DAY'] =  pd.to_datetime(df['DATE'] + timedelta(days=-1))
    
    #Create new df
    cols_to_rename = {"DATE": "LAST_DAY",'VALUE':'VALUE_D-1'}
    df_last = df.drop(['LAST_DAY'], axis=1).rename(index=str, columns=cols_to_rename)
    
    #Merge day-1
    cols_to_merge = ['ENTITY_GROUPS','ENTITY','LAST_DAY','KPI']
    df = df.merge(df_last, on=cols_to_merge, how='left').fillna(0).drop(['LAST_DAY'], axis=1)
    
    #Calc variation in value and %
    df['VARV'] = df['VALUE'].astype(float) - df['VALUE_D-1'].astype(float) 
    df.loc[:,'VARP'] = np.NaN
    df.loc[(df['KPI'].isin(['CONFIRMED','DEATHS','RECOVERED','ACTIVE_CASES'])) & (df['VALUE_D-1'] != 0), 'VARP'] = df['VARV'] * 100 / abs(df['VALUE_D-1'])
    
    #-- Add fields
    df['SCENARIO'] = pd.to_datetime(df['DATE'], format='%m/%d/%y').dt.strftime('%d/%m/%Y')
    df['DATE_ORDER'] = pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y').dt.strftime('%Y%m%d')
    
    #-- Units / Precisions / Sentiments 
    df['KPI'] = df['KPI'].str.replace('_',' ').str.lower().str.capitalize()
    df= pd.merge(df,ref, on=['KPI'], how='left')
    df.loc[df['KPI'] == 'Deaths ratio', 'KPI'] = "Fatality Rate"
    df.loc[df['KPI'] == 'Recovered ratio', 'KPI'] = "Recovery Rate"
    return df.reset_index(drop=True)

db_all = step4(db_conso,ref_tech)
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
db_all.to_csv(output_folder + 'WORLD_DB_ALL.csv',sep=";")
# if use_mongo:
#     bob.mongo.save_df(db_all,'WORLD_DB_ALL',db_src,True)
# db_all

Script execution completed at 13/04/2020 12:26:07. Time: --- 4.24602198600769 secnds ---
Dataframe WORLD_DB_ALL successfully save in database covid-19-dev in MongoDB. Time: --- 37.461220502853394 secnds ---


In [15]:
#--- STEP5 ---: Transform DB_ALL to trend data (output = DB_TREND)
start_time = time.time()
#-> Referentials
ref_tech = pd.read_excel(input_folder + 'REF_WSR.xlsx', sheet_name = 'PARAM_TREND')

def step5(df,ref):
    #-- Melt CALC in rows
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','SCENARIO','DATE','DATE_ORDER','KPI']
    df= df.drop(['VALUE_D-1','UNIT_VALUE','UNIT_VAR','UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP'],axis=1).melt(id_vars=cols_to_keep, value_name='VALUE',var_name='METRIC')
    df['DATE_SCENARIO'] = 'Since beginning'
        
    df_last14 = get_lastdays(df, 14,"Last 14 days")
    df_last30 = get_lastdays(df, 30,"Last 30 days")
    df = pd.concat([df,df_last14,df_last30],axis=0)
    
    #-- Get units and precisions
    df= pd.merge(df,ref, on=['KPI','METRIC'], how='left')
    df.loc[df['KPI'] == 'Deaths ratio', 'KPI'] = "Fatality Rate"
    df.loc[df['KPI'] == 'Recovered ratio', 'KPI'] = "Recovery Rate"
    df = df.reset_index(drop=True)
    return df

db_trend = step5(db_all,ref_tech)
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
db_trend.to_csv(output_folder + 'WORLD_DB_TREND.csv',sep=";")
# if use_mongo:
#     bob.mongo.save_df(db_trend,'WORLD_DB_TREND',db_src,True)
# db_trend.head(5)

Script execution completed at 13/04/2020 12:27:09. Time: --- 20.88958191871643 secnds ---
Dataframe WORLD_DB_TREND successfully save in database covid-19-dev in MongoDB. Time: --- 101.92262101173401 secnds ---


In [16]:
#--- STEP6 ---: Check and validate data
#-> CHECK TIMESERIES
#change yesterday format to match with header
yesterday_c = yesterday.strftime('%m/%d/%y')
if yesterday_c[:1]== '0':
    yesterday_c = yesterday_c[1:].replace('/0','/')

#check if yesterday exist in df
def check_timeseries(date,df,df_name):
    if date in df.columns:
        check = df.agg({date:'sum'}).fillna(0)[0]
    else:
        check = 0
        print(f'Column {date} does not exist in {df_name}.\n')
    return check
    
check_ct = check_timeseries(yesterday_c,cConfirmed, 'CONFIRMED') + check_timeseries(yesterday_c,cConfirmedUS, 'CONFIRMED')
check_dt = check_timeseries(yesterday_c,cDeath, 'DEATHS') + check_timeseries(yesterday_c,cDeathUS, 'DEATHS')
check_rt = check_timeseries(yesterday_c,cRecovery, 'RECOVERED')
check_at = check_ct - check_dt - check_rt
if check_ct != 0:
    check_frt = check_dt / check_ct * 100
    check_rrt = check_rt / check_ct * 100
else:
    check_frt = 0
    check_rrt = 0

#-> CHECK DAILY
#check if yesterday exist in df
def check_timeseries(date_check,df,variable):
    if len(df) != 0 and "Date" in df.columns and variable in df.columns and type(date_check) is date:
        check = df[df['Date'] == date_check.strftime('%m/%d/%y')].agg({variable:'sum'})[0]
    else:
        check = 0
        print(f'No data in Daily\n')
    return check

check_cd = check_timeseries(yesterday,cDaily,'Confirmed')
check_dd = check_timeseries(yesterday,cDaily,'Deaths')
check_rd = check_timeseries(yesterday,cDaily,'Recovered')
check_ad = check_cd - check_dd - check_rd
if check_cd != 0:
    check_frd = check_dd / check_cd * 100
    check_rrd = check_rd / check_cd * 100
else:
    check_frd = 0
    check_rrd = 0
    
#-- CHECK : TIMESERIES vs DAILY
check_c = check_ct - check_cd
check_d = check_dt - check_dd
check_r = check_rt - check_rd
check_a = check_at - check_ad
check_fr = check_frt - check_frd
check_rr = check_rrt - check_rrd

print(f'Data check {yesterday}: TimeSeries - Daily')
print(f'Confirmed: {check_ct} - {check_cd} = {check_c}')
print(f'Deaths: {check_dt} - {check_dd} = {check_d}')
print(f'Recovered: {check_rt} - {check_rd} = {check_r}')
print(f'Active cases: {check_at} - {check_ad} = {check_a}')
print(f'Fatality rate: {check_frt} - {check_frd} = {check_fr}')
print(f'Recovery rate: {check_rrt} - {check_rrd} = {check_rr}')

Data check 2020-04-12: TimeSeries - Daily
Confirmed: 1846670 - 1846680 = -10
Deaths: 114085 - 114090 = -5
Recovered: 421722 - 421722 = 0
Active cases: 1310863 - 1310868 = -5
Fatality rate: 6.177876935240189 - 6.178114237442329 = -0.00023730220214002173
Recovery rate: 22.836890186118797 - 22.83676652154136 = 0.00012366457743695491
